In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig

# Representación digital de la información

Algunas definiciones
- **Señal**: Variación de una propiedad física medible que se usa para transmitir información.
- **Información**: Aquello que permite determinar en un punto del espacio y tiempo el resultado de una elección realizada en otro punto distinto, esto es, aquello que permite responder una pregunta.
- **Señal analógica**: Es la señal en la cual la información es comunicada por el valor de una propiedad que puede variar en forma contínua.
- **Señal digital**: Es la señal en la cual la información es comunicada por la membresía del valor de una propiedad en uno de un número finito de conjuntos posibles.

In [ ]:
params_figura = {
        "figure.dpi":100,
        "figure.autolayout":True,
        "lines.linewidth":1
}

In [ ]:
def fig1():
    t=np.linspace(0,1,10000)
    h=(4+5*t-9*t**2)
    y_am=h*np.sin(2*np.pi*100*t)
    with plt.rc_context(params_figura):
        plt.figure(1,figsize=(10,6))
        plt.clf()
        plt.subplot(2,1,1)
        plt.plot(t,h)
        plt.grid()
        plt.title("Señal analógica (tensión : altura)")
        plt.subplot(2,1,2)
        plt.plot(t,y_am)
        plt.grid()
        plt.title('Señal analógica (amplitud : altura)')
    plt.show()
fig1()

In [ ]:
def fig2():
    def interp_0(xs,n):
        ys = np.repeat(np.zeros_like(xs),n)
        ys[::n]=xs
        return ys
    cod = [1,1,1,1,1,-1,-1,1,1,-1,1,-1,1]
    sps = 500
    fc  = 200
    filt = sig.firwin(20*sps+1,1/(2*sps),window="boxcar",pass_zero=True,fs=1)
    retardo = filt.argmax()
    cod_s = np.convolve(1/filt.max()*interp_0(cod,sps),filt,'full')
    cod_rect = np.concatenate([[0]*(retardo-(sps-1)),np.repeat(cod,sps),[0]*(len(cod_s)+(sps-1)-len(cod)*sps-retardo)])
    t=np.linspace(0,1,len(cod_s))
    portadora = np.sin(2*np.pi*fc*t)
    bpsk = portadora*cod_s
    ks = np.arange(retardo,retardo+len(cod)*sps,sps,dtype=int)
    with plt.rc_context(params_figura):
        plt.figure(2,figsize=(10,3*2))
        plt.clf()
        plt.grid()
        plt.subplot(2,1,1)
        plt.title("Señal digital (2-PAM), secuencia binaria 1111100110101")
        c1 = '#42F'
        c2 = '#F80'
        c3 = '#0004'
        plt.plot(t[ks[0]-sps:ks[-1]+sps//2],cod_s[ks[0]-sps:ks[-1]+sps//2],color=c1)
        plt.plot(t[ks],cod_s[ks],'o',markeredgecolor=c1,markerfacecolor="#0000")
        plt.plot(t[ks[0]-sps:ks[-1]+sps//2],cod_rect[ks[0]-sps:ks[-1]+sps//2],color=c2)
        plt.plot(t[ks],cod_rect[ks],'x',color=c2)
        plt.grid()
        plt.legend(['Sinc','Muestras sinc','Rectangular','Muestras rectangular'])
        plt.subplot(2,1,2)
        plt.title("Señal digital (BPSK pulso sinc), secuencia binaria 1111100110101")
        plt.plot(t[ks[0]-sps//2:ks[-1]+sps//2],portadora[ks[0]-sps//2:ks[-1]+sps//2],color=c3)
        plt.plot(t[ks[0]-sps//2:ks[-1]+sps//2],bpsk[ks[0]-sps//2:ks[-1]+sps//2],color=c1)
        plt.vlines(t[ks],-1.5,1.5,'#F808',linestyles='--')
    plt.show()
fig2()

# Representación digital

- Las señales digitales son equivalentes a números de precisión finita.
- Aunque una señal pueda variar en forma contínua, solo podemos medir con precisión finita pues no es posible conocer perfectamente el valor de una magnitud física.
- Por lo tanto *podemos representar la información medible de una señal analógica mediante una señal digital*.

## Proceso de digitalización

- *Muestreo*: Tomamos muestras de la señal analógica periódicamente para capturar la variación en el tiempo. La frecuencia con que se toman muestras se denomina *frecuencia de muestreo*
- *Cuantización*: Medimos cada muestra por comparación con una escala formada por una secuencia de intervalos.
- *Codificación*: Representamos mediante un código digital la posición de cada muestra en la escala.


In [ ]:
def fn(t):
    return np.sin(2*np.pi*t)
def fig3():
    t = np.linspace(0,1,100)
    y = fn(t)
    ts = np.linspace(0,1,6)
    ys = fn(ts)
    with plt.rc_context(params_figura):
        plt.figure(3,figsize=(10,3))
        plt.plot(t,y)
        plt.stem(ts,ys)
        plt.xlim(0,1)
        plt.title("Muestreo de una señal analógica")
    plt.show()
fig3()


In [ ]:
from collections import namedtuple
ResultadoCuantizacion = namedtuple('ResultadoCuantizacion',['indices','cod','valores','lim_izq','lim_der'])
class Cuantizador(object):
    def __init__(self,niveles):
        self.niveles = niveles
    @property
    def niveles(self):
        return self._niveles
    @niveles.setter
    def niveles(self,valor):
        qdivs = np.array([-1+k*2/valor for k in range(valor+1)])
        self._qdivs = qdivs
        self._qcentros =(qdivs[1:]+qdivs[:-1])/2
    @property
    def qdivs(self):
        return self._qdivs
    @property
    def qcentros(self):
        return self._qcentros
    def cuantiza(self,x):
        x=np.array(x)
        indices = np.zeros(x.shape,"i")
        cod = indices.copy()
        valores = np.zeros(x.shape,self._qcentros.dtype)
        lim_izq = np.zeros(x.shape,self._qdivs.dtype)
        lim_der = lim_izq.copy()
        for k in range(x.size):
            i=max(np.searchsorted(self._qdivs,x.item(k))-1,0)
            indices.itemset(k,i)
            cod.itemset(k,i-len(self._qcentros)//2)
            valores.itemset(k,self._qcentros[i])
            lim_izq.itemset(k,self._qdivs[i])
            lim_der.itemset(k,self._qdivs[i+1])
        return  ResultadoCuantizacion(indices,cod,valores,lim_izq,lim_der)
def fig4():
    qn = Cuantizador(16)
    t = np.linspace(0,1,600)
    ts = np.linspace(0,1,6)
    ys = fn(ts)
    y  = fn(t)
    _,_,yq,_,_ = qn.cuantiza(ys)
    with plt.rc_context(params_figura):
        plt.figure(4,figsize=(10,3))
        plt.plot(t,y,color="#0008")
        plt.plot(ts,ys,'o')
        plt.plot(ts,yq,'rx')
        plt.yticks(qn.qdivs)
        plt.grid(axis='y',which='major')
        plt.xlim(0,1)
        plt.show()
# |-7|-6|-5|-4|-3|-2|-1| 0| 1| 2| 3| 4| 5| 6| 7|
fig4()


In [ ]:
def codigo_bin_signo(x,nbits):
    def digitos(x,nbits):
        for k in range(nbits-1,-1,-1):
            if x&(1<<k):
                yield '1'
            else:
                yield '0'
    return ''.join(digitos(int(x),nbits))
def fig5():
    N =4
    qn = Cuantizador(2**N)
    t = np.linspace(0,1,600)
    y = fn(t)
    ts = np.linspace(0,1,6)
    ys = fn(ts)
    ks,cs,yq,_,_ = qn.cuantiza(ys)
    text = [[k,f'{us:.4f}',uq,f'{n} -> "{codigo_bin_signo(n,N)}"'] for k,(us,uq,n) in enumerate(zip(ys,ks,cs))]
    with plt.rc_context(params_figura):
        plt.figure(5,figsize=(10,6))
        plt.subplot(2,1,1)
        plt.plot(t,y,color="#0008")
        plt.plot(ts,yq,'x')
        plt.yticks(qn.qdivs)
        plt.grid(axis='y',which='major')
        plt.xlim(0,1)
        plt.subplot(2,1,2)
        a = plt.gca()
        a.axis("off")
        plt.table(
            cellText=text,
            colLabels=["Nro. muestra","Tensión","Nr.Intervalo","Codigo"],
            bbox=[0,0,1,1],
            cellLoc="center")
        plt.show()
fig5()


## Conversión digital a analógica.

- Recorremos las muestras digitales según la *frecuencia de muestreo*.
- Modificamos una tensión o corriente de salida para que tome valores proporcionales a cada muestra.
- Usamos un filtro para interpolar entre las muestras y obtener una variación suave.
    - Esto se llama *reconstrucción*


In [ ]:
def fig6(f=1,seno=True,fs=5):
    N = 4
    qn = Cuantizador(2**N)
    tmin = -3
    tmax = 6
    Ns = (tmax-tmin)*fs+1
    ts = np.linspace(tmin,tmax,Ns)
    func = np.sin if seno else np.cos
    ys = func(2*np.pi*f*ts)
    _,_,yq,_,_ = qn.cuantiza(ys)
    interp = 20
    paso = ts[1]-ts[0]
    textra = (paso*(interp-1)/interp)
    t = np.linspace(tmin,tmax+textra,Ns*interp)

    y = np.repeat(yq,20)
    sos = sig.butter(N=5,
                        Wn=fs/2*1.1,
                        btype='low',
                        analog=False,
                        output="sos",
                        fs=fs*interp)
    yf = sig.sosfiltfilt(sos,y)
    fmt = params_figura|{
          "lines.markeredgewidth":3,
          "figure.figsize":[12,6]}
    with plt.rc_context(fmt):
        plt.figure(6)
        plt.subplot(2,1,1)
        plt.plot(t,func(2*np.pi*f*t),label="Original")
        plt.plot(ts,yq,'x',label="Muestras")
        plt.plot(t,y,label="Salida DAC")
        plt.plot(t,yf,'--',label="Filtrado")
        plt.xlim(0,3)
        plt.legend(ncol=4,loc="lower center",bbox_to_anchor=(0.5,1))
        plt.subplot(2,1,2)
        plt.plot(*sig.periodogram(y))
        plt.plot(*sig.periodogram((2,)*20,nfft=len(t),detrend=False))
fig6(f    =  2,
     seno = True,
     fs   = 6)


In [ ]:
fs = 1  # Frecuencia de muestreo
f = 0.3 # Frecuencia de sinusoide
tmin = -3 # extremos del intervalo de tiempo
tmax = 6
Ns = (tmax-tmin)*fs+1 # Número de muestras en el intervalo de tiempo
ts = np.linspace(tmin,tmax,Ns) # Tiempos muestrales
interp = 20
paso = ts[1]-ts[0]
textra = (paso*(interp-1)/interp) # Tiempo extra sobre tmax para permitir interpolación
t = np.linspace(tmin,tmax+textra,Ns*interp) # Tiempos de simulación de señal analógica
xs= np.sin(2*np.pi*f*ts) # Muestras digitales
xdac=np.repeat(xs,interp) # Salida simulada del DAC
x = np.sin(2*np.pi*f*t)   # Señal analógica simulada
plt.plot(ts,xs,'x')
plt.plot(t,x)
plt.plot(t,xdac)


sos = sig.butter(N=5,
                    Wn=fs/2*1.1,
                    btype='low',
                    analog=False,
                    output="sos",
                    fs=fs*interp) # Filtro de reconstrucción
xf = sig.sosfiltfilt(sos,xdac)

plt.plot(t,xf)

plt.xlim(left=0)


## Teorema de Nyquist-Shannon

Es posible reconstruir perfectamente una señal analógica *limitada en ancho de banda* de ancho de banda $B$ a partir de muestras de la misma tomadas con frecuencia de muestreo $f_s$ **si y solo si**

$$ f_s > 2B $$

A la frecuencia límite se la denomina frecuencia de Nyquist

$$f_n=\dfrac{f_s}{2}$$

## Rango dinámico y error de cuantización

El teorema de Nyquist-Shannon considera muestras tomadas con precisión infinita. En la práctica es necesario cuantizar las muestras para poder representarlas de forma digital. En ese caso se introduce una distorsión conocida como *ruido de cuantización* que limita el *rango dinámico* de la señal.


In [ ]:
def fig_x1(nbits_q):
    def mag_dB(mag,min=-100):
        mag = mag.copy()
        piso = 10**(min/20)
        mag[mag<10**(min/20)]=10**(min/20)
        return 20*np.log10(mag)
    qn=Cuantizador(2**nbits_q)
    N=4096
    f=3
    plt.figure(figsize=(10,6),dpi=100)
    t=np.linspace(0,1,N,endpoint=False)
    x=np.sin(2*np.pi*f*t)
    x[x>1]=1
    x[x<-1]=-1
    x_q=qn.cuantiza(x).valores
    plt.subplot(2,1,1)
    plt.plot(t,x)
    plt.plot(t,x_q)
    plt.plot(t,x_q-x)
    plt.subplot(2,1,2)
    F,X_mag=sig.periodogram(x,1/N,detrend=False)
    plt.plot(F,mag_dB(X_mag,-300))
    _,Ruido_mag=sig.periodogram(x_q-x,1/N,detrend=False)
    plt.plot(F,mag_dB(Ruido_mag,-300))
    plt.ylim(bottom=-300)
    plt.show()
fig_x1(12)

## Rango dinámico de ADC *ideal*
Supongamos sin périda de generalidad que la plena escala del ADC es 1V (Vmin -0.5V, Vmax 0.5V).
Considerando una señal sinusoidal de tensión pico a pico de 1V, la relación señal a ruido y distorsión en un ADC *ideal* puede computarse así

$$P_\text{s}=\dfrac{1}{8}$$
$$P_\text{nad}=\dfrac{1}{12\cdot2^{N_\text{b}+1}}$$
$$\text{SINAD}=10\log_{10}{\dfrac{P_\text{s}}{P_\text{nad}}}=10\log_{10}{\dfrac{12\cdot2^{N_\text{b}+1}}{8}}$$
$$\text{SINAD}=10\log_{10}{\left(\dfrac{3}{2}\cdot2^{N_\text{b}+1}\right)}=1,76 + 6,02 N_\text{b}$$

## Número de bits efectivo de un ADC *real*

$$\text{ENOB}=\dfrac{\mathrm{SINAD_{medida}}-1,76}{6,02}$$

## Problema de ejemplo

Dado un sistema con muestreo en radiofrecuencia consistente en una antena, con $T_\text{A}=290\text{ K}$, recibiendo una señal con un nivel de potencia $P_\text{s}=-40\text{ dBm}$. Una cadena de ganancia dominada por un LNA con cifra de ruido $\text{NF}=1.7\text{dB}$, ([adh8411s](https://www.analog.com/en/products/adh8411s.html#product-overview)) con ancho de banda de 3 GHz fijado por un filtro antialias. Determinar las características del conversor analógico-digital necesarias para que la contribución de dicho conversor al ruido sea comparable a la de la porción analógica (antena y cadena de ganacia).
Encontrar un conversor analógico digital comercial adecuado.